# Restoration of Cone Circuit Functionality in the Regenerating Adult Zebrafish Retina 

Evelyn Abraham, Hella Hartmann, Takeshi Yoshimatsu, Tom Baden, Michael Brand





In [4]:
# Import curve fitting package from scipy
from scipy.optimize import curve_fit

# Import required packages
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pylab import cm
import random as rn

from numpy import genfromtxt

#import csv as csv
import pandas as pd

In [5]:
# Function to calculate the exponential with constants a and b
def exponential(x, a, b,c):
    return -a*(np.exp(b*x)-c)

In [6]:
#Import data

my_data = genfromtxt('../private/input/AVG.csv', delimiter=',') #new folder name with /

QI_data = genfromtxt('../private/input/QI.csv',delimiter=',')

rate=0.00109375

extension="png"
folder="output/" 

ar = np.asarray

covBlank=0.0*np.array([[0,0,0],[0,0,0],[0,0,0]])
parsBlank=0.0*np.array([0,0,0])


In [7]:
def rsquared(pars,xdata,ydata,f):
    residuals = ydata- f(xdata, *pars)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)    
    return r_squared

In [8]:
def Eval(ROI=1,results=None,guess1=None,guess2=None,maxfev1=1000,maxfev2=1000,rsqCutOff=0.97,maxParam=100,QIcut=0.7):
    y_rdata=my_data[1:,ROI-1]
    QI=QI_data[ROI,1]

    print("Working on ROI:", ROI)
    print("The QI is ",QI)
    
    if(QI<QIcut):    
        print("QI too low")
        results += [[ROI,QI,0,*parsBlank.tolist(),*parsBlank.tolist()]]

    else:
        success=1

        n=len(y_rdata)

        #time
        x_data=np.linspace(start=0,stop=rate*n,num=n)


        fig = plt.figure()
        ax = fig.add_axes([0, 0, 1, 1])

        ax.scatter(x_data, y_rdata, s=20, color='#00b3b3', label='Data')



        sign = np.sum(y_rdata-y_rdata[0])
        #print(sign)
        if(sign>0):
            print("Type ON because sign=",sign)
            lim = np.argmax(y_rdata)-1
            newIndex = np.argwhere( y_rdata[lim:] < 0.9*np.max(y_rdata))[0][0]  # ON; default 0.9
            offset=newIndex #int(0.09/rate)
            if(guess1):
                guessPars1 = guess1
            else:
                guessPars1 = [19,-1,1]
            
            if(guess2):
                guessPars2 = guess2
            else:
                guessPars2 = [-3000,-5,0]
        else:
            print("Type OFF because sign=",sign)
            lim = np.argmin(y_rdata)-1
            #0.75
            # newIndex = np.argwhere( y_rdata[lim:] > 0.8*np.min(y_rdata))[0][0] 
            newIndex = np.argwhere( y_rdata[lim:] > 0.9*(np.min(y_rdata) - y_rdata[0]) + y_rdata[0] )[0][0] #OFF
            print("The new index",newIndex)
            offset=newIndex #-lim #int(0.4/rate)
            
            if(guess1):
                guessPars1 = guess1
            else:
                guessPars1 = [-14,-4,1]
            
            if(guess2):
                guessPars2=guess2
            else:
                guessPars2 = [300,-3,0]
            



        y_data1 = y_rdata[:lim]
        x_data1 = x_data[:lim]
        ax.scatter(x_data1, y_data1, s=20, color='blue', label='Data')

        # Fit the dummy exponential data
        try:
            pars1, cov1 = curve_fit(f=exponential, xdata=x_data1, ydata=y_data1, p0=guessPars1, bounds=(-np.inf, np.inf),maxfev=maxfev1)
        except:
            try:
                pars1, cov1 = curve_fit(f=exponential, xdata=x_data1, ydata=y_data1, p0=[0, 0, 0], bounds=(-np.inf, np.inf),maxfev=maxfev1)
            except:
                pars1 = parsBlank
                cov1 = covBlank
                success=0 #failed to find the fit

        res = y_data1 - exponential(x_data1, *pars1)

        ax.plot(x_data1, exponential(x_data1, *pars1), linestyle='--', linewidth=2, color='black')

        print(pars1,"\n",cov1)
        rsquared1=rsquared(pars1,x_data1,y_data1,exponential)
        print("rsquared_1",rsquared1)


        y_data2 = y_rdata[lim+offset:]
        x_data2 = x_data[lim+offset:]

        ax.scatter(x_data2, y_data2, s=20, color='red', label='Data')
            
        Δ=np.array([0,0,0])
        pars_cand=parsBlank
        cov_cand=covBlank
        
        for i in range(maxParam):            
            try:
                pars_cand, cov_cand = curve_fit(f=exponential, xdata=x_data2, ydata=y_data2, p0=np.array(guessPars2)+Δ, bounds=(-np.inf, np.inf),maxfev=maxfev2)
            except:
                print("\rFailed Again")
                pass
            
            #Boundary condition
            if(i==0):
                pars2=pars_cand
                cov2=cov_cand
            #Update if the new value is better
            elif(rsquared(pars_cand,x_data2,y_data2,exponential)>rsquared(pars2,x_data2,y_data2,exponential)):
                pars2=pars_cand
                cov2=cov_cand
            
            #If it is a good enough fit, end
            if(rsquared(pars2,x_data2,y_data2,exponential)>rsqCutOff):
                break
                
            #New guess for the parameters     
            #Original: 100,100; -10,10; -10,10
            if(sign<0):
                Δ=[rn.uniform(0,100),rn.uniform(-10,2),rn.uniform(-10,10)]
            else:
                Δ=[rn.uniform(-100,0),rn.uniform(-10,2),rn.uniform(-10,10)]
    
    
        res = y_data2 - exponential(x_data2, *pars2)

        ax.plot(x_data2, exponential(x_data2, *pars2), linestyle='--', linewidth=2, color='black')
        
        plt.xlabel("Time (in seconds)")
        plt.ylabel("Z score (SD) of ROI:"+str(ROI))
              

        print(pars2,"\n",cov2)
        rsquared2=rsquared(pars2,x_data2,y_data2,exponential)
        print("rsquared_2",rsquared2)
        
        fig.savefig(fname=folder+str(ROI)+"."+extension,format=extension, bbox_inches = "tight", dpi=300) 
    
        if(results is not None):
            results += [[ROI,QI,success,exponential(x_data[lim],*pars1), *pars1.tolist(), *pars2.tolist(), rsquared1, rsquared2, exponential(x_data1[0],*pars1),exponential(x_data2[-1],*pars2), sign ]] #y_data1[0],y_data2[-1]]] #cov1.tolist(),pars2.tolist(),cov2.tolist()]
    

### Results

In [ ]:
#ROI has to go from 1 now 

results = [] 
length=len(my_data[1])

for ROI in range(1,length +1,1):
    Eval(ROI,results,maxfev2=1000000,maxParam=15,QIcut=0.4) #specify here alone!


Working on ROI: 1
The QI is  0.0
QI too low
Working on ROI: 2
The QI is  0.0851329
QI too low
Working on ROI: 3
The QI is  0.107223
QI too low
Working on ROI: 4
The QI is  0.255323
QI too low
Working on ROI: 5
The QI is  0.0834635
QI too low
Working on ROI: 6
The QI is  0.0940396
QI too low
Working on ROI: 7
The QI is  0.0
QI too low
Working on ROI: 8
The QI is  0.105575
QI too low
Working on ROI: 9
The QI is  0.063317
QI too low
Working on ROI: 10
The QI is  0.122571
QI too low
Working on ROI: 11
The QI is  0.907933
Type OFF because sign= -3294.3822509999995
The new index 199
[-4.51653658 -5.38097911  1.25463917] 
 [[4.57734292e-04 2.90483078e-04 1.13746606e-04]
 [2.90483078e-04 4.51088955e-03 2.60494082e-04]
 [1.13746606e-04 2.60494082e-04 3.78752147e-05]]
rsquared_1 0.9863083958865304
[ 4.84563709e+02 -4.83068258e+00 -5.59354593e-04] 
 [[ 4.69424291e+02 -1.00988677e+00  1.09253388e-04]
 [-1.00988677e+00  2.18830645e-03 -1.88239478e-07]
 [ 1.09253388e-04 -1.88239478e-07  2.83884393e-

C:\Users\Evelyn\AppData\Local\Temp\ipykernel_10876\3497268888.py:3: RuntimeWarning: overflow encountered in exp
  return -a*(np.exp(b*x)-c)
C:\Users\Evelyn\AppData\Local\Temp\ipykernel_10876\3497268888.py:3: RuntimeWarning: overflow encountered in multiply
  return -a*(np.exp(b*x)-c)
C:\Users\Evelyn\anaconda3\Lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Working on ROI: 44
The QI is  0.930634
Type OFF because sign= -5589.0940040000005
The new index 130
[-7.38764129 -5.10806139  1.15478732] 
 [[3.23371717e-04 1.61649633e-04 4.71970238e-05]
 [1.61649633e-04 9.26488415e-04 5.99412752e-05]
 [4.71970238e-05 5.99412752e-05 8.79901054e-06]]
rsquared_1 0.9959709221724231
[ 1.64108807e+03 -5.55028756e+00 -2.65268646e-04] 
 [[ 5.90924164e+03 -3.60429218e+00  4.55436991e-04]
 [-3.60429218e+00  2.21046072e-03 -2.63898665e-07]
 [ 4.55436991e-04 -2.63898665e-07  6.79381571e-11]]
rsquared_2 0.990977460701203
Working on ROI: 45
The QI is  0.149879
QI too low
Working on ROI: 46
The QI is  0.0933658
QI too low
Working on ROI: 47
The QI is  0.512901
Type ON because sign= 1635.458836231
[-1.91877609  1.3637671   1.48599122] 
 [[0.03362585 0.01608356 0.00743959]
 [0.01608356 0.00773481 0.00358226]
 [0.00743959 0.00358226 0.00166709]]
rsquared_1 0.9724315646950807
[-6.3030573  -0.91851608  0.2615395 ] 
 [[0.01478537 0.00710035 0.00255211]
 [0.00710035 0.014

In [ ]:
pd.DataFrame(results).to_csv(folder+"pythonFit.csv")